In [1]:
import copy
import numpy as np
import time

def parse_board(board):
    """
    Converts a string representation of a Sudoku board into a numpy array, 
    handling various input formats and converting non-digits to zeros.
    
    Args:
        board (str): A string representing a Sudoku board where:
            - Rows are separated by newlines
            - Numbers can be separated by spaces (which are ignored)
            - Non-digit characters are treated as empty cells
            - Each row should contain 9 valid positions after parsing
    
    Returns:
        numpy.ndarray: A 9x9 numpy array where:
            - Digits from input are converted to integers
            - Non-digit characters are converted to 0
            - Spaces are ignored
    
    Process:
        1. Strips leading/trailing whitespace from input string
        2. Splits string into rows at newlines
        3. For each row:
            - Strips whitespace
            - Filters out spaces
            - Converts digits to integers
            - Converts non-digits to 0
        4. Converts resulting list of lists to numpy array
    
    Example:
        Input:
            "1 2 3\n
             . . 4\n
             7 8 9"
        Output:
            array([[1, 2, 3],
                  [0, 0, 4],
                  [7, 8, 9]])
    """
    return np.array([[int(c) if c.isdigit() else 0 for c in row.strip() if c != " "] for row in board.strip().split('\n')])

def print_board(board):
    """
    Prints a Sudoku board in a formatted, readable grid layout with 
    dividing lines to clearly show 3x3 boxes.
    
    Args:
        board (list/ndarray): A 9x9 2D array/list representing a Sudoku board
    
    Output Format:
        - Horizontal lines (- - - - - - - - - - - -) separate 3x3 box rows
        - Vertical bars (|) separate 3x3 box columns
        - Numbers are separated by spaces
        - Each row ends with a newline
        - Extra newline at the end for spacing
    
    Example Output:
        1 2 3 | 4 5 6 | 7 8 9
        4 5 6 | 7 8 9 | 1 2 3
        7 8 9 | 1 2 3 | 4 5 6
        - - - - - - - - - - - -
        2 3 4 | 5 6 7 | 8 9 1
        5 6 7 | 8 9 1 | 2 3 4
        8 9 1 | 2 3 4 | 5 6 7
        - - - - - - - - - - - -
        3 4 5 | 6 7 8 | 9 1 2
        6 7 8 | 9 1 2 | 3 4 5
        9 1 2 | 3 4 5 | 6 7 8
    """
    for i, row in enumerate(board):
        if i % 3 == 0 and i != 0:
            print("- - - - - - - - - - - -")
        for j, val in enumerate(row):
            if j % 3 == 0 and j != 0:
                print("|", end=" ")
            if j == 8:
                print(val)
            else:
                print(str(val) + " ", end="")
    print()


def board_is_solved(board):
    """
    Validates whether a Sudoku board is completely and correctly solved according to all Sudoku rules.
    
    Args:
        board (list): A 9x9 2D list representing a filled Sudoku board
    
    Returns:
        bool: True if the board is a valid solution, False otherwise
    
    Validation Process:
        For each number (1-9), checks three types of constraints:
        1. Row Constraints:
            - Each number must appear exactly once in each row
            - Returns False if number is missing or appears multiple times
        
        2. Column Constraints:
            - Each number must appear exactly once in each column
            - Returns False if number is missing or appears multiple times
        
        3. 3x3 Box Constraints:
            - Each number must appear exactly once in each 3x3 box
            - Returns False if number is missing or appears multiple times
    
    Examples:
        Valid board will contain:
        - Each row: exactly one of each number 1-9
        - Each column: exactly one of each number 1-9
        - Each 3x3 box: exactly one of each number 1-9
    
    Note:
        - Returns False immediately upon finding any violation
        - Must pass all constraints for all numbers to return True
        - Assumes board is completely filled (no zeros/empty cells)
        - Time complexity is O(n^3) where n=9 (fixed for Sudoku)
    """

    for num in range(1, 10):
        # Check rows
        for i in range(9):
            count = 0
            for j in range(9): #cycle through columns of a fixed row
                if board[i][j] == num:
                    count += 1
                if count > 1: #if a number appear more than once in a row
                    return False
            if count < 1: #if a number doesn't appear in a row
                return False
                    
        # check columns
        for j in range(9):
            count = 0
            for i in range(9): #cycle through rows of a fixed column
                if board[i][j] == num:
                    count += 1
                if count > 1: #if a number appear more than once in a row
                    return False
            if count < 1: #if a number doesn't appear in a row
                return False

    # Check boxes
        for box_x in range(1, 3):
            for box_y in range(1, 3):
                count = 0
                for i in range(box_y * 3, box_y * 3 + 3):
                    for j in range(box_x * 3, box_x * 3 + 3):
                        if board[i][j] == num:
                            count += 1
                        if(count > 1): #if a number appears more than one time in a box 3x3
                            return False
                if count < 1: #if a number doesn't appear in a box
                    return False
    return True


def board_is_full(board):
    """
    Checks if a Sudoku board is completely filled (contains no empty cells).
    
    Args:
        board (list): A 9x9 2D list representing a Sudoku board where:
            - 0 represents an empty cell
            - Numbers 1-9 represent filled cells
    
    Returns:
        bool: True if no empty cells (zeros) exist, False otherwise
    
    Process:
        1. Iterates through each cell in the 9x9 grid
        2. Returns False immediately upon finding any zero
        3. Returns True only if no zeros are found
    
    Example:
        >>> board = [[5,3,0],  # Contains zero -> False
                    [6,0,9],
                    [8,9,2]]
        >>> board = [[5,3,4],  # No zeros -> True
                    [6,7,9],
                    [8,9,2]]
    """
    for i in range(9):
        for j in range(9):
            if board[i][j] == 0:
                return False
    return True

In [2]:
def get_first_free_pos(board):
    """
    Finds the coordinates of the first empty cell in a Sudoku board, scanning row by row.
    
    Args:
        board (list): A 9x9 2D list representing a Sudoku board where:
            - 0 represents an empty cell
            - Numbers 1-9 represent filled cells
    
    Returns:
        tuple or None: 
            - tuple (row, col) of first empty cell coordinates if found
            - None if no empty cells exist (board is full)
    
    Process:
        1. Scans board from top-left to bottom-right
        2. Returns coordinates of first zero encountered
        3. Returns None if no zeros found
    """
    
    for i in range(9):
        for j in range(9):
            if board[i][j] == 0:
                return (i,j) #row, column
    
    return None

def get_valid_num_in_position(board, pos):
    """
    Determines all valid numbers that can be placed in a specific position on a Sudoku board
    according to Sudoku rules (row, column, and 3x3 box constraints).
    
    Args:
        board (numpy.ndarray): A 9x9 numpy array representing the Sudoku board
        pos (tuple): A tuple (row, col) indicating the position to check
    
    Returns:
        list: List of integers (1-9) that can legally be placed at the specified position,
              after removing numbers that would violate Sudoku constraints
    
    Process:
        1. Starts with list of all possible numbers [1-9]
        2. Eliminates numbers by checking three constraints:
            a. Row constraint: removes numbers already in the same row
            b. Column constraint: removes numbers already in the same column
            c. 3x3 box constraint: removes numbers already in the same box
        3. Returns remaining valid numbers
    
    Note:
        - Expects numpy array input for efficient array indexing
        - Position tuple is in (row, col) format
        - Empty positions in input board should be 0
        - Returns empty list if no valid numbers exist for the position
    """
    
    list = [1,2,3,4,5,6,7,8,9]
    #check row
    for j in range(9):
        if board[pos[0], j] in list:
            list.remove(board[pos[0], j])
            
    #check column
    for i in range(9):
        if board[i, pos[1]] in list:
            list.remove(board[i, pos[1]])
            
    #check box 3x3
    box_x = pos[1] // 3
    box_y = pos[0] // 3
    for i in range(box_y*3, box_y*3 + 3):
        for j in range(box_x*3, box_x*3 + 3):
            if board[i, j] in list:
                list.remove(board[i, j])
    
    return list                

In [3]:
def solve_board(board, backtracks, forward):
    """
    Recursively solves a Sudoku board using backtracking algorithm, tracking both forward 
    moves and backtracks.
    
    Args:
        board (numpy.ndarray): A 9x9 numpy array representing the Sudoku board where 0s are empty cells
        backtracks (list): Single-element list containing backtrack counter [count]
        forward (list): Single-element list containing forward move counter [count]
    
    Returns:
        bool: True if a valid solution is found, False if no solution exists
    
    Process:
        1. Base cases:
            - If board is full, checks if it's a valid solution
            - If not full, finds first empty position
        
        2. For each valid number at empty position:
            a. Places the number
            b. Increments forward move counter
            c. Recursively tries to solve remaining board
            d. If recursive call succeeds, solution is found
            e. If recursive call fails:
                - Increments backtrack counter
                - Removes the number (backtracks)
                - Tries next valid number
    
    Note:
        - Uses lists for counters to allow modification in recursive calls
        - Forward moves count each attempted placement
        - Backtracks count each time a placement leads to dead end
        - Modifies board in-place
        - Uses depth-first search strategy
        - Returns False if no valid solution exists
        - Performance depends on initial board state and number order
    """
    
    if board_is_full(board):
        return board_is_solved(board)
    else:
        pos = get_first_free_pos(board)
        
    for num in get_valid_num_in_position(board, pos):
        board[pos[0]][pos[1]] = num
        forward[0] += 1
        
        if solve_board(board, backtracks, forward):
            return True
        backtracks[0] += 1
        
        board[pos[0]][pos[1]] = 0
    
    return False

In [4]:
# Example Board
list_board = [
("""
.7. ..4 13.
... 2.7 ..6
..5 .13 .2.
..1 ..2 ...
..2 19. .57
..3 .45 8.2
.1. 378 26.
367 ... 58.
8.9 ..1 .7.
""","easy"),

("""
37. 5.. ..6
... 36. .12
... .91 75.
... 154 .7.
..3 .7. 6..
.5. 638 ...
.64 98. ...
59. .26 ...
2.. ..5 .64
""", "easy"),

    ("""
    ..7..249.
    .....52..
    284.167..
    .49..382.
    ...5....9
    .72.94.31
    ..57389.2
    .6....1..
    .2.6.13.4
    """, "easy"),

    ("""
    ..615...2
    ......7..
    7...84...
    3..9.8561
    .8.......
    ..2..5..8
    ..7...6.3
    53.6..984
    .4...1..7
    """, "medium"),

    ("""
    ..1.....8
    .4.......
    82754....
    ...41..87
    ...7.3.92
    .7..285.6
    49......3
    ..3....69
    218......
    """, "medium"),

    ("""
    71.53.482
    25...4...
    86.97213.
    1..36....
    ..2...6..
    ....91.43
    3....9...
    ..17...26
    4.7...35.
    """, "medium"),

    ("""
    8.6...3..
    .4.1.2.5.
    .925.....
    ....13.4.
    ....5..63
    42..87...
    ..9...781
    .......39
    ..489.5..
    """, "hard"),

    ("""
    .....253.
    61.....2.
    ...3..649
    .....7...
    .45....18
    .9682...3
    3...45.6.
    4..67....
    ..1.9...4
    """, "hard"),

("""
8........
..36.....
.7..9.2..
.5...7...
....45.7.
...1...3.
..1....68
..85...1.
.9....4..
""", "hard"),

    ("""
     .........
     .....3.85 
     ..1.2....
     ...5.7...
     ..4...1..
     .9.......
     5......73
     ..2.1....
     ....4...9
    """, "very hard")
]

In [8]:
difficulties = []
exec_time = []
backtracks_list = []
forwards_list = []

for board_el, difficulty in list_board:
    start = time.time()
    board = parse_board(board_el)
    print("Board:", difficulty)
    print_board(board)
    
    backtracks = [0]
    forward = [0]
    
    solved = solve_board(board, backtracks, forward)
    if solved and board_is_solved(board):
        end = time.time()
        print("Board ", difficulty, " solved: ", "Backtracking steps: ", backtracks, ", Forward steps: ", forward, ", Execution time: ", end - start)
        print("Soluzione: ")
        print_board(board)
        
        difficulties.append(difficulty)
        exec_time.append(end - start)
        backtracks_list.append(backtracks[0])
        forwards_list.append(backtracks[0])
    else:
        print("Unsolvable board")


Board: easy
0 7 0 | 0 0 4 | 1 3 0
0 0 0 | 2 0 7 | 0 0 6
0 0 5 | 0 1 3 | 0 2 0
- - - - - - - - - - - -
0 0 1 | 0 0 2 | 0 0 0
0 0 2 | 1 9 0 | 0 5 7
0 0 3 | 0 4 5 | 8 0 2
- - - - - - - - - - - -
0 1 0 | 3 7 8 | 2 6 0
3 6 7 | 0 0 0 | 5 8 0
8 0 9 | 0 0 1 | 0 7 0

Board  easy  solved:  Backtracking steps:  [594] , Forward steps:  [637] , Execution time:  0.0225064754486084
Soluzione: 
2 7 6 | 9 8 4 | 1 3 5
1 3 8 | 2 5 7 | 9 4 6
9 4 5 | 6 1 3 | 7 2 8
- - - - - - - - - - - -
7 5 1 | 8 3 2 | 6 9 4
4 8 2 | 1 9 6 | 3 5 7
6 9 3 | 7 4 5 | 8 1 2
- - - - - - - - - - - -
5 1 4 | 3 7 8 | 2 6 9
3 6 7 | 4 2 9 | 5 8 1
8 2 9 | 5 6 1 | 4 7 3

Board: easy
3 7 0 | 5 0 0 | 0 0 6
0 0 0 | 3 6 0 | 0 1 2
0 0 0 | 0 9 1 | 7 5 0
- - - - - - - - - - - -
0 0 0 | 1 5 4 | 0 7 0
0 0 3 | 0 7 0 | 6 0 0
0 5 0 | 6 3 8 | 0 0 0
- - - - - - - - - - - -
0 6 4 | 9 8 0 | 0 0 0
5 9 0 | 0 2 6 | 0 0 0
2 0 0 | 0 0 5 | 0 6 4

Board  easy  solved:  Backtracking steps:  [29] , Forward steps:  [75] , Execution time:  0.004007101058959961
S

KeyboardInterrupt: 

In [7]:
for board_el, difficulty in list_board:
    board = parse_board(board_el)
    print("Board:", difficulty)
    hints = 0
    for i in range(9):
        for j in range(9):
            if board[i][j] != 0:
                hints += 1
    print("hints: ", hints, "empty: ",  81 - hints)
    print()

Board: easy
hints:  38 empty:  43

Board: easy
hints:  35 empty:  46

Board: easy
hints:  38 empty:  43

Board: medium
hints:  30 empty:  51

Board: medium
hints:  30 empty:  51

Board: medium
hints:  36 empty:  45

Board: hard
hints:  30 empty:  51

Board: hard
hints:  30 empty:  51

Board: hard
hints:  21 empty:  60

Board: very hard
hints:  17 empty:  64

